In [5]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from scipy.ndimage import gaussian_filter
import numpy as np
from skimage.transform import resize, rotate
from PIL import Image
import nibabel as nib
import os
import cv2
import numpy as np
from numba import cuda
from concurrent.futures import ThreadPoolExecutor
from skimage.transform import resize, rotate

In [2]:
%matplotlib inline

In [6]:
def imgage_path_to_array(image_path):
    if '.dcm' in image_path:
        dicom_data = pydicom.dcmread(image_path)
        return dicom_data.pixel_array
    elif '.nii.gz' in image_path:
        nifti_data = nib.load(image_path)
        return nifti_data.get_fdata()
    else:
        return np.array(Image.open(image_path))

In [16]:
def plot_arr(arr, hide_axis=False, norm = False):
    if norm:
        arr = arr / np.max(np.abs(arr))
    cmap = plt.cm.bwr
    v_min = np.min([np.min(arr),-1])
    v_max = np.max([np.max(arr),1])
    plt.imshow(arr, cmap=cmap, interpolation='nearest', vmin=v_min, vmax=v_max)
    if hide_axis:
        plt.axis('off')
    plt.colorbar() 
    plt.show()

In [23]:

# Load images using opencv which is faster for image manipulations
def load_image(file_path):
    return cv2.imread(file_path, cv2.IMREAD_COLOR)

# Using opencv for rotation and flipping which are more optimized
def rotate_arr(arr, angle):
    rows, cols = arr.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(arr, M, (cols, rows))#, borderMode=cv2.BORDER_REFLECT)

def flip_array_lr(arr):
    return cv2.flip(arr, 1)

# Using opencv for resizing which is more efficient
def resize_image(arr, target_shape):
    try:
        return cv2.resize(arr, (target_shape[1], target_shape[0]))#, interpolation=cv2.INTER_AREA)
    except:
        return cv2.resize(arr, (128, 128))


def generate_frame(image_array, t, max_elongation, width_weight):
    elongation_factor_height = 1.0 + max_elongation * t
    elongation_factor_width = 1.0 + max_elongation * width_weight * t
    resized_image = resize_image(image_array, (int(image_array.shape[0] * elongation_factor_height),
                                               int(image_array.shape[1] * elongation_factor_width)))
    return resized_image

def average_third_dimension(array):
    gray_image = cv2.cvtColor(array, cv2.COLOR_BGR2GRAY)
    return gray_image[:, :, np.newaxis]  # Add the third dimension explicitly


target_side = 128
def process_image(file_path):
    original_image = load_image(file_path)
    sequences = []
    n = 1
    m = 20

    target_shape = (target_side, target_side)
    print(target_shape)
    print(file_path)
    for i in range(n):
        frames = []
        time_wave_start = 2 * (np.random.rand() - 0.5)
        time_wave_end = (2 + time_wave_start) + 2 * (np.random.rand() - 0.5)
        time_wave = np.sin(np.linspace(time_wave_start * np.pi, time_wave_end * np.pi, m))
        max_elongation = np.random.rand() #* 0.3
        width_weight = np.random.rand() #* 0.3
        if np.random.rand() > 0.5:
            original_image = flip_array_lr(original_image)
        rot_angle = (np.random.rand() - 0.5) * 10
        original_image = rotate_arr(original_image, rot_angle)

        for j in range(m):
            frame = generate_frame(original_image, time_wave[j], max_elongation, width_weight)
            frame = resize_image(frame, target_shape)
            frame = average_third_dimension(frame)
            frames.append(frame)

        sequences.append(frames)
    return sequences


def process_directory(directory_path):
    all_sequences = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_image, os.path.join(directory_path, filename))
                   for filename in os.listdir(directory_path) if filename.endswith('.png')]
        for future in futures:
            all_sequences.extend(future.result())

    np.save(f'synthetic_output_sequences_{target_side}.npy', np.array(all_sequences))
    print(f"Sequences saved successfully. (shape: { np.array(all_sequences).shape}) ")


directory_path = "synthetic_data"
process_directory(directory_path)


(128, 128)
synthetic_data/segmentation_mask_seed_618193_size_349.png
(128, 128)
synthetic_data/segmentation_mask_seed_896157_size_914.png
(128, 128)
synthetic_data/segmentation_mask_seed_169716_size_879.png
(128, 128)
synthetic_data/segmentation_mask_seed_339368_size_32.png
(128, 128)
synthetic_data/segmentation_mask_seed_126379_size_909.png
(128, 128)
synthetic_data/segmentation_mask_seed_720948_size_112.png
(128, 128)
synthetic_data/segmentation_mask_seed_666343_size_386.png
(128, 128)
synthetic_data/segmentation_mask_seed_629169_size_953.png
(128, 128)
synthetic_data/segmentation_mask_seed_859485_size_921.png
(128, 128)
synthetic_data/segmentation_mask_seed_704064_size_441.png
(128, 128)
synthetic_data/segmentation_mask_seed_54619_size_672.png
(128, 128)
synthetic_data/segmentation_mask_seed_667038_size_242.png
(128, 128)
synthetic_data/segmentation_mask_seed_667630_size_769.png
(128, 128)
synthetic_data/segmentation_mask_seed_288336_size_885.png
(128, 128)
synthetic_data/segmentati

In [24]:
import numpy as np

# Load the data
file_path = "synthetic_output_sequences_128.npy"  # Replace with your actual .npy file path
data = np.load(file_path)

# Check if the data is at least 2-dimensional
if data.ndim < 2:
    raise ValueError("Data must be at least 2-dimensional.")

# Shuffle the data along the first axis
np.random.shuffle(data)

# Calculate the split index for a 90:10 split
split_idx = int(0.9 * data.shape[0])

# Split the data into training and validation sets
train_data = data[:split_idx]
val_data = data[split_idx:]

# Save the split datasets
np.save("synthetic_train_data_128.npy", train_data)
np.save("synthetic_val_data_128.npy", val_data)

print("Synthetic train and validation data saved successfully.")
train_data.shape


Synthetic train and validation data saved successfully.


(1800, 20, 128, 128, 1)